# Cross Decomposition Example

Original Model: 

$ \min \> x_1 + 2x_2 +3y + 4w\\
s.t. \> \>
x_1 + x_2 + y \geq 6\\
-3x_1 + 2x_2 + w \geq 7\\
x,y,w \geq 0\\
$


In [25]:
using JuMP
using Gurobi

originalModel = Model(solver = GurobiSolver())

@variable(originalModel, x[1:2]>=0)
@variable(originalModel, y>=0)
@variable(originalModel, w>=0)

@constraint(originalModel, x[1]+x[2]+y>=6)
@constraint(originalModel, -3x[1]+2x[2]+w>=7)

@objective(originalModel, Min, x[1]+2x[2]+3y+4w)

solve(originalModel)

x = getindex(originalModel,:x)
y = getindex(originalModel,:y)
w = getindex(originalModel,:w)

println("x = ", getvalue(x))
println("y = ", getvalue(y))
println("w = ", getvalue(w))

Academic license - for non-commercial use only
Optimize a model with 2 rows, 4 columns and 6 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 7e+00]
Presolve removed 0 rows and 2 columns
Presolve time: 0.05s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.4955000e+00   1.880625e+00   0.000000e+00      0s
       1    1.1000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.05 seconds
Optimal objective  1.100000000e+01
x = [1.0, 5.0]
y = 0.0
w = 0.0


Reformulated Model:
    
$\min \> x_{A1} + 2x_{A2}+ 3y + 4w\\
s.t. \> \>
x_{A1}+x_{A2}+ y \geq 6\\
-3x_{B1}+2x_{B2}+w\geq 7\\
x_{A1}-x_{B1} = 0\\
x_{A2}-x_{B2} = 0\\
$

In [26]:
using JuMP
using Gurobi
reformulatedModel = Model(solver = GurobiSolver())

@variable(reformulatedModel, xA[1:2]>=0)
@variable(reformulatedModel, xB[1:2]>=0)
@variable(reformulatedModel, y>=0)
@variable(reformulatedModel, w>=0)

@objective(reformulatedModel, Min, xA[1] + 2xA[2] + 3y + 4w)

@constraint(reformulatedModel, xA[1] + xA[2] + y >= 6)
@constraint(reformulatedModel, -3xB[1] + 2xB[2] + w >= 7)
@constraint(reformulatedModel, dual1, xA[1] - xB[1] == 0)
@constraint(reformulatedModel, dual2, xA[2] - xB[2] == 0)

solve(reformulatedModel)

x = getindex(reformulatedModel,:xA)
y = getindex(reformulatedModel,:y)
w = getindex(reformulatedModel,:w)

println("x = ", getvalue(x))
println("y = ", getvalue(y))
println("w = ", getvalue(w))

Academic license - for non-commercial use only
Optimize a model with 4 rows, 6 columns and 10 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 7e+00]
Presolve removed 2 rows and 4 columns
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.4955000e+00   1.880625e+00   0.000000e+00      0s
       1    1.1000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds
Optimal objective  1.100000000e+01
x = [1.0, 5.0]
y = 0.0
w = 0.0


Bender's Master Problem:

$\min \> x_1 + 2x_2 + \theta_1 + \theta_2 \\ 
s.t. \> \> x, \theta \geq 0\\
$


Bender's SP 1:

$ \min \> 3y \\
s.t. \> \> x_1 + x_2 + y \geq 6\\
x_1 - \bar x_1 = 0 \\
x_2 - \bar x_2 = 0 \\
y \geq 0 \\
$


Bender's SP 2:

$ \min \> 4w \\
s.t. \> \> -3x_1 + 2x_2 + w \geq 7\\
x_1 - \bar x_1 = 0 \\
x_2 - \bar x_2 = 0 \\
w \geq 0 \\
$

In [27]:
using JuMP
using Gurobi
BMP = Model(solver = GurobiSolver())
BSP1 = Model(solver = GurobiSolver())
BSP2 = Model(solver = GurobiSolver())

@variable(BMP, x[1:2] >= 0)
@variable(BMP, θ[1:2] >= 0)

@objective(BMP, Min, x[1]+x[2]+θ[1]+θ[2])


@variable(BSP1, xbar[1:2])
@variable(BSP1, x[1:2]>=0)
@variable(BSP1, y >=0)

@constraint(BSP1, x[1] + x[2] + y >= 6)
@constraint(BSP1, dual1, x[1]-xbar[1]==0)
@constraint(BSP1, dual2, x[2]-xbar[2]==0)

@objective(BSP1, Min, 3y)

@variable(BSP2, xbar[1:2])
@variable(BSP2, x[1:2]>=0)
@variable(BSP2, w >=0)

@constraint(BSP2, -3x[1] + x[2] + w >= 7)
@constraint(BSP2, dual1, x[1]-xbar[1]==0)
@constraint(BSP2, dual2, x[2]-xbar[2]==0)

@objective(BSP2, Min, 4w)

4 w

Lagrange Master Problem:

$ \max \> \eta \\
s.t. \> \> \eta \leq z_{k1} + z_{k2} + \lambda_1 (x_{A1}-x_{B1}) + \lambda_2 (x_{A2}-x_{B2}) \\
$

Lagrange SP 1:

$ \min \> x_{A1} + 2x_{A2} + 3y +\lambda_1x_{A1} + \lambda_2x_{A2}\\
   s.t. \> \> x_{A1} + x_{A2} + y \geq 6 \\
   x_A,y \geq 0 \\
$

Lagrange SP 2:

$ \min \> 4w - \lambda_1x_{B1} - \lambda_2x_{B2}\\
   s.t. \> \> x_{B1} + x_{B2} + y \geq 6 \\
   x_B,y \geq 0 \\
$

In [77]:
LMP = Model(solver = GurobiSolver())
LSP1 = Model(solver = GurobiSolver())
LSP2 = Model(solver = GurobiSolver())

@variable(LMP, η<=0)
@variable(LMP, zk[1:2])
@variable(LMP, λ[1:2]>=0)
@variable(LMP, xA[1:2]>=0)
@variable(LMP, xB[1:2]>=0)

@constraint(LMP, η<= zk[1]+zk[2]+λ[1]*(xA[1]-xB[1])+λ[2]*(xA[2]-xB[2]))

@objective(LMP, Max, η)

@variable(LSP1, xA[1:2]>=0)
@variable(LSP1, y>= 0)
@variable(LSP1, λ[1:2]>=0)

@constraint(LSP1, xA[1]+xA[2]+y>=6)
@objective(LSP1, Min, xA[1]+2xA[2]+3y+λ[1]*xA[1]+λ[2]*xA[2])

@variable(LSP2, xB[1:2]>=0)
@variable(LSP2, w>= 0)
@variable(LSP2, λ[1:2]>=0)

@constraint(LSP2,λ[1:2].<=0)
@constraint(LSP2, -3xB[1]+xB[2]+w>=7)
@objective(LSP2, Min, 4w - λ[1]*xB[1]-λ[2]*xB[2])

:Min

Solve benders master to get $\bar x$ values:

In [78]:
solve(BMP)
x = getindex(BMP,:x)
print(getvalue(x))

Optimize a model with 0 rows, 4 columns and 0 nonzeros
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  0.000000000e+00
[0.0, 0.0]

Solution: $x_1, x_2 = (0,0)$

Now fix these values into BSPs:

In [79]:
x = getindex(BSP1, :xbar)

setlowerbound(x[1],0)
setupperbound(x[1],0)

setlowerbound(x[2],0)
setupperbound(x[2],0)

x = getindex(BSP2, :xbar)

setlowerbound(x[1],0)
setupperbound(x[1],0)

setlowerbound(x[2],0)
setupperbound(x[2],0)

0

Now that the values from the BMP have been fixed into the BSPs, we can get the duals, $\mu$s, and objectives, $\theta$s from the BSPs:

In [80]:
solve(BSP1)
dual1 = getindex(BSP1,:dual1)
dual2 = getindex(BSP1,:dual2)

μ11 = getdual(dual1)
μ12 = getdual(dual2)
θk1 = getobjectivevalue(BSP1)

solve(BSP2)
dual1 = getindex(BSP2,:dual1)
dual2 = getindex(BSP2,:dual2)

μ21 = getdual(dual1)
μ22 = getdual(dual2)
θk2 = getobjectivevalue(BSP2)

Optimize a model with 3 rows, 5 columns and 7 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 6e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  1.800000000e+01
Optimize a model with 3 rows, 5 columns and 7 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [4e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+00, 7e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  2.800000000e+01


28.0

With this information from the BSPs, we can make a cut from the BSP to the LMP such that:

$z_k \leq \theta_k + \sum\lambda_i \bar x_i$

In [81]:
print(LSP1)
solve(LSP1)
λ = getindex(LSP1, :λ)
x = getindex(LSP1, :xA)
y = getindex(LSP1, :y)

xMP = getindex(LMP, :xA)

setlowerbound(xMP[1],getvalue(x[1]))
setupperbound(xMP[1],getvalue(x[1]))
setlowerbound(xMP[2],getvalue(x[2]))
setupperbound(xMP[2],getvalue(x[2]))

print(LSP2)
solve(LSP2)
λ = getindex(LSP2, :λ)
x = getindex(LSP2, :xB)
y = getindex(LSP2, :w)

xMP = getindex(LMP, :xB)

setlowerbound(xMP[1],getvalue(x[1]))
setupperbound(xMP[1],getvalue(x[1]))

setlowerbound(xMP[2],getvalue(x[2]))
setupperbound(xMP[2],getvalue(x[2]))

print(LMP)

Min xA[1]*λ[1] + xA[2]*λ[2] + xA[1] + 2 xA[2] + 3 y
Subject to
 xA[1] + xA[2] + y ≥ 6
 xA[i] ≥ 0 ∀ i ∈ {1,2}
 λ[i] ≥ 0 ∀ i ∈ {1,2}
 y ≥ 0
Academic license - for non-commercial use only
Optimize a model with 1 rows, 5 columns and 3 nonzeros
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 6e+00]
Presolve removed 0 rows and 2 columns
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.88510251e+01  0.00000000e+00  0.00e+00 0.00e+00  1.55e+01     0s
   1   1.11440493e+01  4.17311311e+00  0.00e+00 0.00e+00  1.74

In [82]:
xbar = getindex(BSP1, :xbar)
@constraint(LMP, zk[1] <= θk1+λ[1]*getvalue(xbar[1])+λ[2]*getvalue(xbar[2]))
xbar = getindex(BSP2, :xbar)
@constraint(LMP, zk[2] <= θk2+λ[1]*getvalue(xbar[1])+λ[2]*getvalue(xbar[2]))
print(LMP)

Max η
Subject to
 zk[1] ≤ 18
 zk[2] ≤ 28
 -λ[1]*xA[1] - λ[2]*xA[2] + λ[1]*xB[1] + λ[2]*xB[2] + η - zk[1] - zk[2] ≤ 0
 zk[i] ∀ i ∈ {1,2}
 λ[i] ≥ 0 ∀ i ∈ {1,2}
 η ≤ 0
 6.000000000000084 ≤ xA[1] ≤ 6.000000000000084
 6.562803661294217e-14 ≤ xA[2] ≤ 6.562803661294217e-14
 0 ≤ xB[1] ≤ 0
 10.6642651943477 ≤ xB[2] ≤ 10.6642651943477


In [84]:
solve(LMP)
print(getvalue(getindex(LMP,:λ)))

Optimize a model with 2 rows, 9 columns and 2 nonzeros
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [7e-14, 1e+01]
  RHS range        [2e+01, 3e+01]
Presolve removed 2 rows and 9 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.01 seconds
Optimal objective -0.00000000e+00
[0.0, 0.0]

To make a cut from the LSPs to the BMP, first we solve the LSPs for the objectives, $z_k$s, and the duals, $\lambda$s:

In [85]:
solve(LSP1)
λ = getindex(LSP1,:λ)
zk1 = getobjectivevalue(LSP1)
λ11 = getvalue(λ[1])
λ12 = getvalue(λ[2])

#solve(LSP2)
λ = getindex(LSP2, :λ)
zk2 = getobjectivevalue(LSP2)
λ21 = getvalue(λ[1])
λ22 = getvalue(λ[2])

Optimize a model with 1 rows, 5 columns and 3 nonzeros
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 6e+00]
Presolve removed 0 rows and 2 columns
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.88510251e+01  0.00000000e+00  0.00e+00 0.00e+00  1.55e+01     0s
   1   1.11440493e+01  4.17311311e+00  0.00e+00 0.00e+00  1.74e+00     0s
   2   6.28147788e+00  5.97708258e+00  0.00e+00 0.00e+00  7.61e-02     0s
   3   6.00028214e+00  5.99997774e+00  0.00e+00 0.00e+00  7.61e-05     0s
   4   6.00000028e+00  5.

0.0